# 高频因子

## 摘要

本部分主要探讨基于高频数据的因子构建。


## 理论模型

### 订单偏斜(Order Imbalance)

确定了交易方向后(方法参见附录: 交易方向划分算法), 对每个交易日计算如下的 Order Imbalance 因子:
$$
IMBAL_t=\frac{\sum\limits_{b=1}^{B} VOL_{b,t}-\sum\limits_{s=1}^{S} VOL_{s,t}}{\sum\limits_{b=1}^{B} VOL_{b,t}+\sum\limits_{s=1}^{S} VOL_{s,t}}
$$

其中, $VOL_{b,t}$ 表示 t 日第 b 笔划分为买方发起的成交量, $VOL_{s,t}$ 表示 t 日第 s 笔划分为卖方发起的成交量. 总之, IMBAL 因子度量的是买方发起的成交量和卖方发起的成交量的差除以总成交量. 该因子来自于 \cite{Chung-2010}.


### 知情交易概率(Probability of Informed Trading, PIN)

Probability of Informed Trading (PIN) 的概念最早是由 \cite{cit-Easley-1997} 引入的, 相关的文献还有 [Easley-2002]_, [Easley-2010]_. 此处的 PIN 来自于 [Easley-2002]_  建立的市场微观结构模型, 假设有三种市场参与者: 做市商(Market Maker), 知情交易者(Informed Trader)以及不知情交易者(Uninformed Trader), 做市商观察交易行情序列, 其试图推断出交易是由知情交易者发起的还是由不知情交易者发起的, 据此做市商给出最新的买卖报价. 对此建立概率模型:
1. 在每个交易日开始之前, 假设一个消息事件发生的概率为 $\alpha$, 消息事件最初只由知情交易者获得, 知情交易者可以根据该消息事件得到关于股票未来价格变动的信号, 然后入场交易. 如果该交易日没有消息事件发生, 则该交易日的所有交易者为不知情交易者.
2. 在一个消息事件发生的条件下, 假设该消息为利空消息的概率为 $\delta$, 则为利好消息的概率是 $1-\delta$
3. 做市商在该交易日的每个时点设置其买卖报价. 在消息事件发生日, 来自于知情交易者的订单到达率为 $\mu$, 而来自于不知情交易者的买卖订单到达率分别为 $\epsilon_b, \epsilon_s$. 在非消息事件发生日, 所有的订单均来自不知情交易者, 所有买卖订单到达率分别为 $\epsilon_b, \epsilon_s$.

该模型的图示如下:

![Easley et al. 知情交易模型](./images/EasleyTradingModel.png)

做市商并不清楚每笔交易是由知情交易者还是不知情交易者发起的, 但他可以根据行情数据来估计该订单由知情交易者发起的概率. 简单的说, 如果做市商观察到大致相等数量的买卖订单, 则大概率上该交易日是非消息事件日. 然而, 如果其观察到买入订单多于卖出订单, 则可以推断该日发生了利好消息. 如果只是简单的统计, 则该模型和 Order Imbalance 因子没有明显的区别. 构建模型的优势在于可以根据过去一段时间的交易行情数据推断出知情交易发生的概率. 假设非知情交易者发出的买卖订单流是独立的 Poisson 过程, 则其对数似然函数为:
$$
L\left((B_t, S_t)_{t=1}^{T}|\theta\right)=\sum\limits_{t=1}^{T}\left[-\epsilon_b-\epsilon_s+M_t(\ln x_b+\ln x_s)+B_t\ln(\mu+\epsilon_b)+S_t\ln(\mu+\epsilon_s)\right]+\sum\limits_{t=1}^{T}\ln\left[\alpha(1-\delta)e^{-\mu}x_s^{S_t-M_t}x_b^{-M_t}+\alpha\delta e^{-\mu}x_b^{S_t-M_t}x_s^{-M_t}+(1-\alpha)x_s^{S_t-M_t}x_b^{B_t-M_t}\right]
$$
其中, $B_t$ 是 t 日由买方发起的成交量, $S_t$ 是 t 日由卖方发起的成交量, $M_t=\min(B_t,S_t)+\max(B_t,S_t)/2$, $x_s=\epsilon_s/(\mu+\epsilon_s), x_b=\epsilon_b/(\mu+\epsilon_b)$, $\theta=(\mu,\epsilon_b,\epsilon_s,\alpha,\delta)$ 是模型参数.

可以使用最大似然估计方法估计出模型参数, 然后定义 PIN 因子为:
$$
PIN=\frac{\alpha\mu}{\alpha\mu+\epsilon_b+\epsilon_s}
$$
其中, $\alpha\mu+\epsilon_b+\epsilon_s$ 是所有订单的到达率, $\alpha\mu$ 是知情交易订单的到达率.

[Easley-2002]_, [Hwang-2010]_ 指出 PIN 因子倾向于和规模, 流动性以及波动性因子相关, 同时为了衡量 PIN 因子中是否包含了不同于传统因子的独特信息, 需要对 PIN 因子做一些中性化的处理.


### 大额交易的异常成交量(Abnormal Volume in Large Trades, ALT)

这个因子来源于 [Tong-2009]_, 其背后的思想是认为拥有独有消息的知情交易者倾向于呈现更激进的交易行为, 而寻找该行为的一个线索是大额交易中的异常成交量. 该因子的定义为:
1. 用过去一年的交易数据计算成交量 30%, 60%, 90% 的分位数.
2. 将成交量超过 90% 分位数的交易记为大额交易.
3. 统计过去一个月的大额交易成交量之和, 并除以过去一年的大额交易成交量之和作为 ALT 因子.

ALT 因子的一个重要假设是大额交易代表了知情交易行为, 但当前金融市场中, 市场直连通道(Direct Market Access, DMA), 算法交易的兴起, 以及多样性的交易渠道使得交易者可以很好的伪装自己的交易行为, 这会使该因子的有效性减弱.

ALT 因子还有一些变体, 第一个称为: 大额交易比例(Percent of Large Trades, PLT), 即过去一个月的大额交易成交量之和除以过去一个月的总成交量. 第二个称为: RALT(Residual ALT), 即 ALT 关于规模, 波动性, 流动性因子中性化后的残差因子.

## 实证分析

### 因子计算

数据为 A 股的 Level2 指标数据. 来自于 Wind 量化研究数据库的表: 中国A股Level2指标(AShareL2Indicators).

首先设置参数并准备数据

In [16]:
# coding=utf-8
import time
import sys
import os
import datetime as dt
import warnings
warnings.filterwarnings("ignore")
sys.path.append("C:\\Users\\hushuntai\\svn\\python")

import numpy as np
import pandas as pd
import statsmodels.api as sm
from matplotlib.pylab import mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['axes.unicode_minus'] = False
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
# %gui qt4
# %matplotlib notebook
import matplotlib.dates as mdate

from QuantStudio.FactorDataBase.WindDB2 import WindDB2
WDB = WindDB2()
WDB.connect()

IDs = ["000001.SZ", "600749.SH"]
#IDs = WDB.getID(index_id="全体A股", is_current=False)
StartDT = dt.datetime(2018, 1, 4)
EndDT = dt.datetime(2018, 6, 30, 23, 59, 59, 999999)
FT = WDB.getTable("中国A股Level2指标")
DTs = FT.getDateTime(start_dt=StartDT, end_dt=EndDT)
Data = FT.readData(factor_names=["主买总量(手)", "主卖总量(手)", "大买总量(手)", "大卖总量(手)"], ids=IDs, dts=DTs)
WDB.disconnect()

0

**订单偏斜因子(IMBAL)**

In [2]:
IMBAL = (Data["主买总量(手)"]-Data["主卖总量(手)"])/Data.sum(axis=0)

**知情交易概率因子(PIN)**

这里我们使用 iminuit 模块来进行极大似然估计.

In [3]:
T = 60# 训练窗口
ijB, ijS, ijM = 0, 0, 0
ijmu, ijepsilon_s, ijepsilon_b, ijalpha, ijdelta = 1, 1, 1, 0.2, 0.5

def NegtiveLogLikelihood(mu, epsilon_b, epsilon_s, alpha, delta):
    x_s = epsilon_s/(mu+epsilon_s)
    x_b = epsilon_b/(mu+epsilon_b)
    LL = np.sum(-epsilon_b - epsilon_s + ijM*(np.log(x_b)+np.log(x_s))
                + ijB*np.log(mu+epsilon_b) + ijS*np.log(mu+epsilon_s))
    LL += np.sum(np.log(alpha*(1-delta)*np.exp(-mu)*x_s**(ijS-ijM)*x_b**(-ijM)
                        + alpha*delta*np.exp(-mu)*x_b**(ijS-ijM)*x_s**(-ijM)
                        + (1-alpha)*x_s**(ijS-ijM)*x_b**(ijB-ijM)))
    return -LL

from iminuit import Minuit
from QuantStudio.Tools.DateTimeFun import getMonthLastDateTime
TargetDTs = getMonthLastDateTime(list(Data.major_axis))
EndInds = [Data.major_axis.searchsorted(iDT) for iDT in TargetDTs]
PIN = np.full((len(TargetDTs), len(IDs)), np.nan)
for j, jID in enumerate(IDs):
    jB = Data["主买总量(手)"].iloc[:, j].values
    jS = Data["主卖总量(手)"].iloc[:, j].values
    for i, iDT in enumerate(TargetDTs):
        iEndInd = EndInds[i]
        iStartInd = max((0, iEndInd-T+1))
        ijB = jB[iStartInd:iEndInd+1]
        ijS = jS[iStartInd:iEndInd+1]
        ijMask = (pd.notnull(ijB) & pd.notnull(ijS))
        ijB, ijS = ijB[ijMask], ijS[ijMask]
        if ijB.shape[0]==0: continue
        ijScale = max((1, np.max(ijB+ijS)))
        ijB, ijS = ijB/ijScale, ijS/ijScale
        ijM = np.minimum(ijB, ijS)+np.maximum(ijB, ijS)/2
        ijMdl = Minuit(NegtiveLogLikelihood, pedantic=False, print_level=0, errordef=0.5, 
                       mu=ijmu, epsilon_b=ijepsilon_b, epsilon_s=ijepsilon_s, alpha=ijalpha, delta=ijdelta,
                       error_mu=0.1, error_epsilon_b=0.1, error_epsilon_s=0.1, error_alpha=0.1, error_delta=0.1,
                       limit_mu=(0,None), limit_epsilon_b=(0,None), limit_epsilon_s=(0,None), limit_alpha=(0,1), limit_delta=(0,1))
        ijFmin, ijtheta = ijMdl.migrad()
        ijmu, ijepsilon_s, ijepsilon_b, ijalpha, ijdelta = ijtheta[0]["value"], ijtheta[1]["value"], ijtheta[2]["value"], ijtheta[3]["value"], ijtheta[4]["value"]
        PIN[i, j] = ijalpha*ijmu/(ijalpha*ijmu+ijepsilon_b+ijepsilon_s)
PIN = pd.DataFrame(PIN, index=TargetDTs, columns=IDs)

**大额交易的异常成交量(ALT)**

In [7]:
DenominatorWinLen = 240# 分母的窗口长度
NumeratorWinLen = 21# 分子的窗口长度
LargeTradeData = Data.iloc[2:].sum(axis=0)
TargetDTs = LargeTradeData.index[DenominatorWinLen-1:]
ALT = np.full((len(TargetDTs), len(IDs)), np.nan)
for i, iDT in enumerate(TargetDTs):
    iData = LargeTradeData.iloc[i:i+DenominatorWinLen, :].values
    ALT[i, :] = np.nansum(iData[-NumeratorWinLen:, :], axis=0)/np.nansum(iData, axis=0)
ALT[np.isinf(ALT)] = np.nan
ALT = pd.DataFrame(ALT, index=TargetDTs, columns=IDs)

储存因子数据

In [8]:
from QuantStudio.FactorDataBase.HDF5DB import HDF5DB
HDB = HDF5DB()
HDB.connect()
HDB.writeData(pd.Panel({"IMBAL":IMBAL, "PIN":PIN, "ALT":ALT}), "HFFactor", if_exists="replace")

0

### 因子测试

#### IC 测试

In [18]:
from QuantStudio.Tools.QtGUI.QtGUIFun import showOutput
from QuantStudio.Tools.DateTimeFun import getMonthLastDateTime
from QuantStudio.FactorDataBase.HDF5DB import HDF5DB
from QuantStudio.FactorDataBase.CustomDB import FactorCacheFT
from QuantStudio.HistoryTest.HistoryTestModel import HistoryTestModel
from QuantStudio.HistoryTest.SectionTest.IC import IC
from QuantStudio.HistoryTest.SectionTest.Portfolio import QuantilePortfolio

# 创建自定义因子库
MainFT = FactorCacheFT("MainFT")
FT = HDB.getTable("ElementaryFactor")
MainFT.addFactors(factor_table=FT, factor_names=["复权收盘价"], args={})
MainFT.addFactors(factor_table=HDB.getTable("HFFactor"), factor_names=["IMBAL", "PIN", "ALT"], args={})
MainFT.setDateTime(FT.getDateTime(ifactor_name="复权收盘价", start_dt=dt.datetime(2017, 1, 1), end_dt=dt.datetime(2018, 1, 1)))
MainFT.setID(FT.getID(ifactor_name="复权收盘价"))

# 创建回测模型
Model = HistoryTestModel()
Model.Modules.append(IC(factor_table=MainFT))# IC 测试
Model.Modules.append(QuantilePortfolio(factor_table=MainFT))# 分位数组合测试

# 设置模型参数
Model.setArgs()

# 运行模型
TestDateTimes = MainFT.getDateTime()
Model.run(test_dts=TestDateTimes)

# 查看结果
Output = Model.output()
showOutput(Output)

==========历史回测==========
1. 初始化耗时 : 0.61
2. 循环计算

100%|██████████████████████████████████████████| 18/18 [00:02<00:00,  7.44it/s]


耗时 : 2.42
3. 结果生成耗时 : 0.01
总耗时 : 3.05


## 附录

### 交易方向划分算法

很多因子的构建基于对交易方向的确认, 交易方向指的是每一笔交易是由买方还是卖方发起的. 交易方向的信息是不公开的, 所以只能根据公开的行情数据进行估计.

**Tick Test**

Tick Test 相对简单: 如果当前交易的成交价高于上一笔交易的成交价, 则认为当前交易是买方发起的; 反之当前交易的成交价低于上一笔交易成交价, 则认为当前交易是卖方发起的; 特别的, 当前交易成交价和上一笔交易成交价持平, 则认定当前交易方向和上一笔交易方向相同. 如下图所示:

![Tick Test 交易方向划分示例](./images/TickTest示例.png)

**Lee-Ready 算法**

Lee-Ready 算法来自于 Lee 和 Ready [1991], Lee-Ready 算法认为单纯的成交价信息不足以准确的划分交易方向, 结合买卖盘报价数据后可以提高准确率. 方法为: 如果当前成交价高于买卖中间价, 则认为当前交易是买方发起的; 反之低于中间价, 则划分为卖方发起的; 特别的, 如果成交价刚好等于中间价, 则使用 Tick Test 算法来划分. 如下图所示:

![Tick Test 交易方向划分示例](./images/TickTest示例.png)

# References

(<a id="cit-Chung-2010" href="#call-Chung-2010">?</a>) !! _This reference was not found in biblio.bib _ !!

(<a id="cit-Easley-1997" href="#call-Easley-1997">?</a>) !! _This reference was not found in biblio.bib _ !!

